查看cuda版本 cuda>=10.0

In [1]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


安装mxnet和gluonnlp

In [2]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
!pip install --upgrade gluonnlp

     |████████████████████████████████| 256kB 8.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470046 sha256=28cf4c012b67e0f655ca5cbc9b214fd0213d98559e0bd2e12a1417a7754bd1c9
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


查看硬件信息gpu

In [4]:
!nvidia-smi

Mon Jul 13 00:15:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

挂载google云端硬盘

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


根据自己路径进行修改

In [6]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [7]:
cd NLP/

/content/gdrive/My Drive/Colab Notebooks/NLP


In [8]:
#导入相关库
import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
from bert import data
import random

加载模型

In [10]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)
bert_large, vocabulary = nlp.model.get_model('bert_24_1024_16',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
bert_classifier = nlp.model.BERTClassifier(bert_large, num_classes=2, dropout=0.1)
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)
filename="./large_2.params"
nlp.utils.load_parameters(bert_classifier,filename,ctx=ctx)

load a sentence to infer

In [11]:

field_separator = nlp.data.Splitter('\t')
data_inference_row = nlp.data.TSVDataset(filename='test.tsv',
                field_separator=field_separator)

bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
max_len = 128
all_labels = ["0", "1"]
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_inference=data_inference_row.transform(transform)
batch_size = 1
data_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_inference],
                      batch_size=batch_size,
                      shuffle=True)


结果

In [12]:
data_loader = mx.gluon.data.DataLoader(data_inference, batch_sampler=data_sampler)
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(data_loader):
  token_ids = token_ids.as_in_context(ctx)
  valid_length = valid_length.as_in_context(ctx)
  segment_ids = segment_ids.as_in_context(ctx)
  label = label.as_in_context(ctx)
  out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
  print(data_inference_row[0][0])
  print('label: ',data_inference_row[0][1])
  if (out.argmax(axis=1))[0]==1:
    print('inference: 1')
  else:
    print('inference: ')

Another fantastic offering from the Monkey Island team and though it was a long time coming and had to survive the departure of Ron Gilbert it's another worthy installment. My only gripe is that it was a little short seeming in comparison to the previous two, though that might be because of a glorious lack of disk-swapping. Roll on MI4.
label:  1
inference: 1
